In [73]:
# Improve bounding to find nameplate

In [74]:
# Libraries
import pytesseract                      # Optical Character Recognition
from PIL import Image                   # Image Accessing
import cv2                              # Image Modification
import os                               # Directory handling
import pandas as pd                     # Excel sheets
import xlsxwriter                       # Modifying excel sheets
import numpy as np

# Tesseract Setup
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\Elena.Justo\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'

In [75]:
# Target folder
dir = r"sites\rosemeadow"

In [76]:
# Create dataframe file to serve as UI when converted into excel file
user_df = pd.DataFrame(data=[])

# Debug
user_df

""


In [77]:
# Getting file paths
fileNames = []
for item in os.listdir(dir):
    fileNames.append(item)

filePaths = []
for item in os.listdir(dir):
    filePaths.append(dir + "\\" + item)
    
# Debug
print(fileNames)
print(filePaths)

['IMG_7240.JPG', 'IMG_7244.JPG', 'IMG_7246.JPG', 'IMG_7248.JPG', 'IMG_7250.JPG', 'IMG_7264.JPG', 'IMG_7266.JPG', 'IMG_7268.JPG', 'IMG_7270.JPG', 'IMG_7272.JPG', 'IMG_7274.JPG', 'IMG_7276.JPG', 'IMG_7278.JPG', 'IMG_7280.JPG', 'IMG_7282.JPG', 'IMG_7284.JPG', 'IMG_7286.JPG', 'IMG_7288.JPG', 'IMG_7290.JPG', 'IMG_7292.JPG', 'IMG_7294.JPG', 'IMG_7296.JPG', 'IMG_7298.JPG', 'IMG_7300.JPG', 'IMG_7302.JPG', 'IMG_7304.JPG', 'IMG_7306.JPG', 'IMG_7308.JPG', 'IMG_7313.JPG', 'IMG_7315.JPG', 'IMG_7317.JPG', 'IMG_7319.JPG', 'IMG_7321.JPG', 'IMG_7323.JPG', 'IMG_7325.JPG', 'IMG_7327.JPG', 'IMG_7329.JPG', 'IMG_7331.JPG', 'IMG_7333.JPG', 'IMG_7335.JPG', 'IMG_7343.JPG', 'IMG_7345.JPG', 'IMG_7347.JPG', 'IMG_7349.JPG', 'IMG_7350.JPG', 'IMG_7352.JPG', 'IMG_7354.JPG', 'IMG_7356.JPG', 'IMG_7358.JPG', 'IMG_7360.JPG', 'IMG_7362.JPG', 'IMG_7364.JPG', 'IMG_7366.JPG', 'IMG_7368.JPG', 'IMG_7370.JPG', 'IMG_7372.JPG', 'IMG_7374.JPG', 'IMG_7376.jpg', 'IMG_7378.JPG', 'IMG_7382.jpg', 'IMG_7384.jpg', 'IMG_7386.jpg', 'IMG_73

In [78]:
# Save file names into UI
user_df["File Name"] = fileNames

# Debug
user_df

,File Name
0,IMG_7240.JPG
1,IMG_7244.JPG
2,IMG_7246.JPG
3,IMG_7248.JPG
4,IMG_7250.JPG
...,...
312,IMG_7931.jpg
313,IMG_7933.jpg
314,IMG_7935.JPG
315,IMG_7937.JPG


In [79]:
user_df.insert(1, "Image Preview", None, False)

user_df.insert(2, "OCR Results", None, False)

user_df.insert(3, "Mask 1", None, False)

user_df.insert(4, "Bounding View", None, False)

In [80]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter("boundTesting.xlsx", engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
user_df.to_excel(writer, sheet_name='Sheet1')

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# Setting column widths
worksheet.set_column(first_col=0, last_col=0, width=10)      # Index column width
worksheet.set_column(first_col=1, last_col=1, width=15)      # Filename column width
worksheet.set_column(first_col=2, last_col=8, width=50)      # Image columns width

# Embed Raw Images
for image in fileNames:
    worksheet.set_row(row=fileNames.index(image)+1, height=100)                     # Setting row height
    worksheet.embed_image(fileNames.index(image)+1, 2, dir + "\\" + str(image))     # Insert an image.

In [81]:
# Create temp folder to store pre-processing of images
try:
    os.mkdir("temp")
except FileExistsError:
    print("File already exists")

File already exists


In [82]:
# Clean temp folder
for files in os.listdir("temp"):
    os.remove("temp\\"+files)

# OCR Results
results  = []

# Pre-Process Images
for paths in filePaths:

    image = cv2.imread(paths)
    
    print(fileNames[filePaths.index(paths)])

    lower = np.array([0,0,0]) 
    upper = np.array([250,250,170])

    # Define the masked area
    mask = cv2.inRange(image, lower, upper)

    cv2.imwrite("temp\\"+ "mask1_" +fileNames[filePaths.index(paths)], mask)
    worksheet.embed_image(filePaths.index(paths)+1, 4, "temp\\"+ "mask1_" +fileNames[filePaths.index(paths)])

    # Find contours
    contMode = cv2.RETR_TREE

    # Get image contours
    cnts = cv2.findContours(mask, contMode, cv2.CHAIN_APPROX_SIMPLE) 
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    # Draw contours onto image
    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)
        if h > 200 and w > 20:
            roi = mask[y:y+h, x:x+h]
            box = cv2.rectangle(mask, (x, y), (x+w, y+h), (255, 0, 0))

    cv2.imwrite("temp\\"+ "box_" +fileNames[filePaths.index(paths)], box)
    worksheet.embed_image(filePaths.index(paths)+1, 6, "temp\\"+ "box_" +fileNames[filePaths.index(paths)])

# Close the Pandas Excel writer and output the Excel file.
writer.close()

IMG_7240.JPG
IMG_7244.JPG
IMG_7246.JPG
IMG_7248.JPG
IMG_7250.JPG
IMG_7264.JPG
IMG_7266.JPG
IMG_7268.JPG
IMG_7270.JPG
IMG_7272.JPG
IMG_7274.JPG
IMG_7276.JPG
IMG_7278.JPG
IMG_7280.JPG
IMG_7282.JPG
IMG_7284.JPG
IMG_7286.JPG
IMG_7288.JPG
IMG_7290.JPG
IMG_7292.JPG
IMG_7294.JPG
IMG_7296.JPG
IMG_7298.JPG
IMG_7300.JPG
IMG_7302.JPG
IMG_7304.JPG
IMG_7306.JPG
IMG_7308.JPG
IMG_7313.JPG
IMG_7315.JPG
IMG_7317.JPG
IMG_7319.JPG
IMG_7321.JPG
IMG_7323.JPG
IMG_7325.JPG
IMG_7327.JPG
IMG_7329.JPG
IMG_7331.JPG
IMG_7333.JPG
IMG_7335.JPG
IMG_7343.JPG
IMG_7345.JPG
IMG_7347.JPG
IMG_7349.JPG
IMG_7350.JPG
IMG_7352.JPG
IMG_7354.JPG
IMG_7356.JPG
IMG_7358.JPG
IMG_7360.JPG
IMG_7362.JPG
IMG_7364.JPG
IMG_7366.JPG
IMG_7368.JPG
IMG_7370.JPG
IMG_7372.JPG
IMG_7374.JPG
IMG_7376.jpg
IMG_7378.JPG
IMG_7382.jpg
IMG_7384.jpg
IMG_7386.jpg
IMG_7388.jpg
IMG_7392.JPG
IMG_7394.JPG
IMG_7396.jpg
IMG_7398.jpg
IMG_7400.jpg
IMG_7402.jpg
IMG_7404.jpg
IMG_7406.jpg
IMG_7408.JPG
IMG_7410.jpg
IMG_7412.jpg
IMG_7414.jpg
IMG_7416.jpg
IMG_7418.JPG